<a href="https://colab.research.google.com/github/AlexKostjuk/DZ_5/blob/master/dz_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
import kagglehub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist



In [3]:
# Завантаження MNIST без one-hot
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
# Створюємо базову модель EfficientNetV2
base_model = tf.keras.applications.EfficientNetV2S(
    include_top=True,
    weights='imagenet',
    input_shape=(384, 384, 3)
)


In [5]:
# Модель із масштабуванням у шарах
model = models.Sequential([
    layers.Input(shape=(28, 28)),           # Оригінальний розмір MNIST
    layers.Reshape((28, 28, 1)),           # Додаємо канал
    layers.Resizing(384, 384),             # Масштабування в моделі
    layers.Lambda(lambda x: tf.tile(x, [1, 1, 1, 3])),  # Перетворюємо в RGB
    layers.Lambda(lambda x: tf.keras.applications.efficientnet_v2.preprocess_input(x)),
    base_model,
    layers.Dense(512, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

In [6]:
# Компіляція
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [7]:
# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=2,
    restore_best_weights=True
)

In [ ]:
# Навчання з меншим batch_size
history = model.fit(
    x_train, y_train,
    epochs=6,
    batch_size=128,  # Зменшено для економії пам’яті
    validation_data=(x_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/6


In [ ]:
# Оцінка та вивід
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'\nТочність на тестовому наборі: {test_acc:.4f}')